# Chapter 9 - GAN

## ch9.1

### GAN 기본 모델 구현하기

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


#### 하이퍼파라미터 설정

In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128 #노이즈의 크기

#### 플레이스홀더

- GAN도 비지도학습 이므로 Y를 사용하지 않음
- 실제 이미지와 가짜이미지 사용, 가짜이미지는 노이즈에서 생성

In [3]:
X = tf.placeholder(tf.float32, [None, n_input]) # 실제이미지
Z = tf.placeholder(tf.float32, [None, n_noise]) # 가짜이미지

#### 생성자 신경망에 사용할 변수들 설정

- 첫 벌쨰 가중치와 편향은 은닉층으로 출력
- 두 번째 가중치와 편향은 출력층에 사용할 변수. 두 번째 가중치의 변수 크기는 실제 이미지의 크기와 같아야한다. (n_input, 28*28 = 784)

In [4]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden],stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

#### 구분자 신경망에 사용할 변수들 설정

- 구분자는 진짜와 얼마나 가까운가를 판단 0~1사이 값 출력

- 하나의 스칼라값을 출력하도록 구성

In [5]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden],stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

**실제이미지 판별하는 구분자 신경망과 생성한 이미지를 판별하는 구분자 신경망은 같은 변수를 사용해야한다.**

#### 생성자 신경망 구성

- 무작위 생성 노이즈를 받아 가중치와 편향으로 은닉층을 만들고 은닉층에서 실제 이미지와 같은 크기의 결괏값을 출력함

In [6]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z,G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output

#### 구분자 신경망 구성

- 0~1 사이의 스칼라값 하나를 출력하도록 하고, 이를 위한 활성화 함수로 sigmoid를 사용?

In [7]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output

#### 무작위 노이즈를 만들어주는 간단한 유틸리티 함수

In [8]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size,n_noise))

#### 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만들고, 이 G가 만든 가짜이미지와 진짜 이미지 X를 각각 구분자에 넣어 입력한 이미지가 진짜인지를 판별하도록함

In [9]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

#### 손실값 구하기
- 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값(`D_real`)
- 생성자가 만든 이미지를 구분자가 진짜라고 판단하도록 하는 손실값(`D_gene`)

두가지가 필요하다.

#### 가짜를 가짜로 진짜를 진짜로 판별하기 위한 손실 값

> **loss_D = tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))**

- `D_real` 은 1에 가까워야하고, `D_gene`는 0에 가까워야한다.
- `D_real`과 `1에서 D_gene를 뺀 값`을 더한 값을 손실값으로 하여, 이값을 최대화하면 된다.

In [10]:
loss_D = tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))

#### 가짜를 진짜라고 판별하게 하기 위한 손실 값

>**loss_G = tf.reduce_mean(tf.log(D_gene))**

- 가짜 이미지 판별값 `D_gene`를 1에 가깝게 만들어야한다.최대화 시켜야한다.


In [11]:
loss_G = tf.reduce_mean(tf.log(D_gene))

#### GAN에서의 손실값
- GAN학습은 `loss_D`와 `loss_G`를 모두 최대화 시키는 것이다.
- `loss_D`와 `loss_G`는 연관되어있어 항상 같이 증가하진않는다. 경쟁관계이기 때문

In [12]:
D_var_list = [D_W1,D_b1,D_W2,D_b2]
G_var_list = [G_W1,G_b1,G_W2,G_b2]

#### 변수 최적화 함수 구성
- 손실값을 최대화시켜야하지만 최적화함수는 minimize뿐이므로 음수부호를 붙인다.

In [13]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)

#### 학습시작

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0,0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D,loss_D],feed_dict={X: batch_xs, Z:noise})
        _, loss_val_G = sess.run([train_G,loss_G],feed_dict={Z:noise})
        
    print('Epoch:', '%04d' % epoch,'D loss: {:.4}'.format(loss_val_D),'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch +1)%10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1,sample_size, figsize=(sample_size,1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i],(28,28)))
            
        plt.savefig('ch9_samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches = 'tight')
        plt.close(fig)


Epoch: 0000 D loss: -0.5156 G loss: -1.994
Epoch: 0001 D loss: -0.3799 G loss: -2.294
Epoch: 0002 D loss: -0.07814 G loss: -3.44
Epoch: 0003 D loss: -0.7668 G loss: -1.405
Epoch: 0004 D loss: -0.3404 G loss: -1.825
Epoch: 0005 D loss: -0.5167 G loss: -2.024
Epoch: 0006 D loss: -0.2263 G loss: -2.794
Epoch: 0007 D loss: -0.2521 G loss: -2.797
Epoch: 0008 D loss: -0.2103 G loss: -2.88
Epoch: 0009 D loss: -0.1797 G loss: -3.053
Epoch: 0010 D loss: -0.4085 G loss: -2.365
Epoch: 0011 D loss: -0.6239 G loss: -2.124
Epoch: 0012 D loss: -0.3544 G loss: -2.263
Epoch: 0013 D loss: -0.4876 G loss: -1.756
Epoch: 0014 D loss: -0.4495 G loss: -2.276
Epoch: 0015 D loss: -0.3941 G loss: -2.309
Epoch: 0016 D loss: -0.4014 G loss: -2.462
Epoch: 0017 D loss: -0.3193 G loss: -2.737
Epoch: 0018 D loss: -0.4471 G loss: -2.587
Epoch: 0019 D loss: -0.5257 G loss: -2.175
Epoch: 0020 D loss: -0.3471 G loss: -2.54
Epoch: 0021 D loss: -0.5269 G loss: -2.204
Epoch: 0022 D loss: -0.7153 G loss: -2.046
Epoch: 0023 D

#### 결과
![000](https://user-images.githubusercontent.com/43382559/74710061-1c1b9b80-5264-11ea-8817-ae2be188c7cc.png)
![009](https://user-images.githubusercontent.com/43382559/74710064-1d4cc880-5264-11ea-9f50-b4af2cc9a8f0.png)
![019](https://user-images.githubusercontent.com/43382559/74710067-1de55f00-5264-11ea-906e-142f1e573a88.png)
![029](https://user-images.githubusercontent.com/43382559/74710068-1de55f00-5264-11ea-875d-2fa91eeab7d7.png)
![039](https://user-images.githubusercontent.com/43382559/74710069-1e7df580-5264-11ea-9235-0268fff87dea.png)
![049](https://user-images.githubusercontent.com/43382559/74710070-1e7df580-5264-11ea-8bb2-b63479b04376.png)
![059](https://user-images.githubusercontent.com/43382559/74710071-1f168c00-5264-11ea-9c2b-8f7e652918f3.png)
![069](https://user-images.githubusercontent.com/43382559/74710074-1faf2280-5264-11ea-9c14-6300f9cc4cdf.png)
![079](https://user-images.githubusercontent.com/43382559/74710077-1faf2280-5264-11ea-9396-97d998a167a7.png)
![089](https://user-images.githubusercontent.com/43382559/74710079-2047b900-5264-11ea-9789-3636e4f7c785.png)
![099](https://user-images.githubusercontent.com/43382559/74710080-2047b900-5264-11ea-9a97-f3d0a96543c6.png)
